# 3. LangChain Streaming - Respuestas en Tiempo Real

## Objetivos de Aprendizaje
- Comprender qué es el streaming y cuándo usarlo
- Implementar streaming con LangChain
- Manejar chunks de datos en tiempo real
- Construir interfaces de usuario reactivas

## ¿Qué es el Streaming?

El streaming permite recibir la respuesta del modelo **token por token** conforme se genera, en lugar de esperar a que termine completamente. Esto mejora significativamente la experiencia de usuario en aplicaciones interactivas.

### Ventajas del Streaming:
- **Percepción de velocidad**: El usuario ve progreso inmediato
- **Mejor UX**: Interfaces más reactivas e interactivas  
- **Engagement**: Mantiene la atención del usuario
- **Debugging**: Permite ver el proceso de generación

### Casos de Uso Ideales:
- Chatbots y asistentes conversacionales
- Generación de contenido largo
- Aplicaciones web interactivas
- Demostraciones en vivo

In [6]:
# Importar bibliotecas necesarias
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage

import os
import time

print("Bibliotecas importadas correctamente para streaming")

Bibliotecas importadas correctamente para streaming


In [7]:
# Configuración del modelo con streaming habilitado
try:
    llm = ChatOpenAI(
        base_url="https://models.github.ai/inference",  #descubri que aqui  cambia de url base  
        api_key=os.getenv("GITHUB_TOKEN"),               
        model = "openai/gpt-4o-mini",               
        temperature=0.7,
        streaming=True  # Sin streaming
       
    )
    # llm = ChatOpenAI(
    #      base_url=os.getenv("OPENAI_BASE_URL"),
    #      api_key=os.getenv("GITHUB_TOKEN"),
    #      model="gpt-4o",
    #      streaming=True,  # ¡Importante: habilitar streaming!
    #      temperature=0.7
    #  )
    # llm = ChatOpenAI(
    #     base_url="https://models.github.ai/inference",  #descubri que aqui  cambia de url base  
    #     api_key=os.getenv("GITHUB_TOKEN"),               
    #     model = "openai/gpt-4.1",               # modelo DeepSeek
    #     temperature=0.7,
    #     streaming=True, 
    #     max_tokens=150
    # )
    
    print("✓ Modelo configurado con streaming habilitado")
    print(f"Modelo: {llm.model_name}")
    print(f"Streaming: {llm.streaming}")
    
except Exception as e:
    print(f"✗ Error en configuración: {e}")
    print("Verifica las variables de entorno")

✓ Modelo configurado con streaming habilitado
Modelo: openai/gpt-4o-mini
Streaming: True


## Streaming Básico

El método `.stream()` devuelve un generador que produce chunks de texto conforme se generan.

In [10]:
# Ejemplo básico de streaming
def streaming_basico():
    prompt = "Cuéntame una historia breve sobre los beneficios de el oregano "
    
    print("=== STREAMING EN TIEMPO REAL ===")
    print("Generando respuesta...")
    print("-" * 50)
    
    try:
        # stream() devuelve un generador de chunks
        for chunk in llm.stream([HumanMessage(content=prompt)]):
            # Imprimir cada chunk sin nueva línea
            print(chunk.content, end="", flush=True)
            time.sleep(0.01)  # Pequeña pausa para simular streaming visual
            
        print("\n" + "-" * 50)
        print("✓ Streaming completado")
        
    except Exception as e:
        print(f"✗ Error en streaming: {e}")

# Ejecutar streaming básico
streaming_basico()

=== STREAMING EN TIEMPO REAL ===
Generando respuesta...
--------------------------------------------------
Había una vez en un pequeño pueblo llamado Herbalia, donde los habitantes eran conocidos por su amor a las plantas y las especias. En el centro del pueblo, había una hermosa huerta que siempre estaba llena de aromas y colores. Entre todas las hierbas que allí crecían, el orégano era la estrella. 

Doña Clara, la sabia herbolaria del pueblo, siempre decía: “El orégano no solo da sabor a nuestros platillos, sino que también trae consigo muchos beneficios”. Un día, un grupo de niños curiosos se acercó a ella y le preguntó qué hacía tan especial a esta hierba.

Doña Clara sonrió y comenzó a contarles sobre las maravillas del orégano. “Primero”, dijo, “es un gran aliado para nuestra salud. Tiene propiedades antioxidantes que ayudan a combatir los radicales libres y a mantener nuestro cuerpo joven y fuerte. Además, es un excelente antiinflamatorio, perfecto para aliviar esos pequeños do

In [17]:

import sys

import re
from itertools import cycle
from datetime import datetime




# =============================================================================
# EJERCICIO 1: INDICADORES DE PROGRESO
# =============================================================================

def ejercicio1_spinner_simple():
    """Spinner básico que rota mientras llegan chunks"""
  
    spinner = cycle(['|', '/', '-', '\\'])
    prompt = "rwsumido en 30 palabras :Explica los beneficios del oregano para la salud"
    
    print("=== EJERCICIO 1A: SPINNER SIMPLE ===")
    print("Generando respuesta...\n")
    
    response_text = ""
    try:
        
        for chunk in llm.stream([HumanMessage(content=prompt)]):
            # Mostrar spinner
            sys.stdout.write(f'\r{next(spinner)} Procesando...')
            sys.stdout.flush()
            
            response_text += chunk.content
            time.sleep(0.05)
            print(chunk.content, end="", flush=True)
            
        # Limpiar spinner y mostrar resultado
        sys.stdout.write('\r✓ Completado!   \n')
        print(f"\nRespuesta completa:\n{response_text}\n")
        
    except Exception as e:
        print(f"\nError: {e}")
ejercicio1_spinner_simple()        


=== EJERCICIO 1A: SPINNER SIMPLE ===
Generando respuesta...

✓ Completado!   l orégano aporta beneficios antioxidantes, antimicrobianos y antiinflamatorios. Mejora la digestión, refuerza el sistema inmunológico, combate infecciones y contribuye a la salud respiratoria. Sus compuestos ayudan a prevenir enfermedades y promueven el bienestar general.

Respuesta completa:
El orégano aporta beneficios antioxidantes, antimicrobianos y antiinflamatorios. Mejora la digestión, refuerza el sistema inmunológico, combate infecciones y contribuye a la salud respiratoria. Sus compuestos ayudan a prevenir enfermedades y promueven el bienestar general.



In [19]:
#modificado 
import sys
import time
import threading
from itertools import cycle

def spinner_task(stop_event):
    spinner = cycle(['|', '/', '-', '\\'])
    while not stop_event.is_set():
        sys.stdout.write(f'\r{next(spinner)} Procesando...')
        sys.stdout.flush()
        time.sleep(0.1)
    sys.stdout.write('\r' + ' ' * 20 + '\r')  # limpia la línea del spinner

def ejercicio1_spinner_simple():
    prompt = "resumen en 30 palabras :Explica los beneficios del oregano para la salud"
    print("=== EJERCICIO 1A: SPINNER SIMPLE ===")
    print("Generando respuesta...\n")
    response_text = ""
    stop_event = threading.Event()
    spinner_thread = threading.Thread(target=spinner_task, args=(stop_event,))
    spinner_thread.start()

    try:
        # Recibe e imprime chunks normalmente (sin spinner aquí)
        for chunk in llm.stream([HumanMessage(content=prompt)]):
            print(chunk.content, end="", flush=True)
            response_text += chunk.content
            time.sleep(0.05)
        stop_event.set()
        spinner_thread.join()
        print("\n✓ Completado!")
        print(f"\nRespuesta completa:\n{response_text}\n")
    except Exception as e:
        stop_event.set()
        spinner_thread.join()
        print(f"\nError: {e}")

ejercicio1_spinner_simple()


=== EJERCICIO 1A: SPINNER SIMPLE ===
Generando respuesta...

                    égano ofrece propiedades antioxidantes, antiinflamatorias y antimicrobianas. Favorece la digestión, fortalece el sistema inmunológico, protege contra infecciones y contribuye a la salud cardiovascular, gracias a sus compuestos como carvacrol y timol.
✓ Completado!

Respuesta completa:
El orégano ofrece propiedades antioxidantes, antiinflamatorias y antimicrobianas. Favorece la digestión, fortalece el sistema inmunológico, protege contra infecciones y contribuye a la salud cardiovascular, gracias a sus compuestos como carvacrol y timol.



In [22]:


def ejercicio1_barra_progreso():
    """Barra de progreso visual basada en longitud de texto generado"""
    
    prompt = "resume en 20 palabras:Describe el proceso de fotosíntesis en las plantas"
    print("=== EJERCICIO 1B: BARRA DE PROGRESO ===")
    
    response_text = ""
    last_percent = 0
    bar_length = 30
    
    try:
        for chunk in llm.stream([HumanMessage(content=prompt)]):
            response_text += chunk.content
            
            # Progreso estimado basado en cantidad de caracteres
            total_estimate = 30  # puedes ajustar según lo largo que esperas que sea la respuesta
            progress = min(len(response_text) / total_estimate, 1.0)
            
            filled_length = int(bar_length * progress)
            bar = '█' * filled_length + '░' * (bar_length - filled_length)
            percent = int(progress * 100)
            
            # Solo actualizar si hay un cambio real en porcentaje
            if percent != last_percent:
                sys.stdout.write(f'\r[{bar}] {percent}% ({len(response_text)} chars)')
                sys.stdout.flush()
                last_percent = percent
            
            time.sleep(0.03)
        
        # Completa la barra al final
        sys.stdout.write('\r[' + '█'*bar_length + '] 100% (' + str(len(response_text)) + ' chars)\n')
        sys.stdout.flush()
        
        print(f"\n\nRespuesta final:\n{response_text}\n")
        print("✅ Barra de progreso completada!\n")
        
    except Exception as e:
        print(f"\nError: {e}")

ejercicio1_barra_progreso()


=== EJERCICIO 1B: BARRA DE PROGRESO ===
[██████████████████████████████] 100% (117 chars)


Respuesta final:
La fotosíntesis es el proceso donde las plantas convierten luz solar, agua y dióxido de carbono en glucosa y oxígeno.

✅ Barra de progreso completada!



## Comparación: Streaming vs No-Streaming

Veamos la diferencia en experiencia de usuario entre ambos enfoques.

In [9]:
# Comparación entre streaming y no-streaming
def comparar_streaming():

    # Modelo sin streaming
    llm_no_stream = ChatOpenAI(
        base_url="https://models.github.ai/inference",  #descubri que aqui  cambia de url base  
        api_key=os.getenv("GITHUB_TOKEN"),               
        model = "openai/gpt-4o-mini",               
        temperature=0.7,
        streaming=False  # Sin streaming
       
    )
    llm = ChatOpenAI(
        base_url="https://models.github.ai/inference",  #descubri que aqui  cambia de url base  
        api_key=os.getenv("GITHUB_TOKEN"),               
        model = "openai/gpt-4o-mini",               
        temperature=0.7,
        streaming=False  # Sin streaming
       
    )
    
    prompt = "resumido en 150 palabras :Escribe un párrafo sobre beneficions del oregano"
    
    print("=== COMPARACIÓN: STREAMING vs NO-STREAMING ===\\n")
    
    # 1. Sin streaming
    print("1. SIN STREAMING:")
    print("-" * 20)
    print("Esperando respuesta completa...")
    
    start_time = time.time()
    try:
        response = llm_no_stream.invoke([HumanMessage(content=prompt)])
        end_time = time.time()
        
        print(f"\\n[Respuesta recibida después de {end_time - start_time:.2f} segundos]")
        print(response.content)
        
    except Exception as e:
        print(f"Error: {e}")
    
    print("\\n" + "="*60 + "\\n")
    
    # 2. Con streaming
    print("2. CON STREAMING:")
    print("-" * 18)
    print("Respuesta en tiempo real:")
    
    start_time = time.time()
    try:
        for chunk in llm.stream([HumanMessage(content=prompt)]):
            print(chunk.content, end="", flush=True)
            time.sleep(0.03)  # Simular pausa para efecto visual
        
        end_time = time.time()
        print(f"\\n\\n[Streaming completado en {end_time - start_time:.2f} segundos]")
        
    except Exception as e:
        print(f"Error: {e}")
    
    print("\\n" + "="*60)
    print("OBSERVACIONES:")
    print("- Sin streaming: El usuario espera sin feedback")
    print("- Con streaming: El usuario ve progreso inmediato")
    print("- Mejor percepción de velocidad con streaming")
    print("- Streaming es especial para respuestas largas")

# Ejecutar comparación
comparar_streaming()

=== COMPARACIÓN: STREAMING vs NO-STREAMING ===\n
1. SIN STREAMING:
--------------------
Esperando respuesta completa...
\n[Respuesta recibida después de 2.90 segundos]
El orégano es una hierba aromática que ofrece múltiples beneficios para la salud. Su alto contenido de antioxidantes ayuda a combatir el estrés oxidativo, protegiendo las células del daño. Además, posee propiedades antimicrobianas que pueden contribuir a la lucha contra infecciones bacterianas y fúngicas. El orégano es rico en compuestos antiinflamatorios, lo que puede aliviar síntomas de enfermedades inflamatorias y mejorar la salud digestiva al estimular la producción de jugos gástricos. También se le atribuyen efectos beneficiosos en la salud respiratoria, al actuar como expectorante natural. Su uso en la cocina no solo realza el sabor de los platillos, sino que también promueve una alimentación más saludable. En resumen, el orégano es una valiosa adición tanto en la gastronomía como en la medicina natural, ofreciendo

## Implementación de un Chatbot Simple con Streaming

Creemos un chatbot básico que demuestre el streaming en un contexto práctico.

In [ ]:
# Chatbot simple con streaming
def chatbot_streaming():
    print("=== CHATBOT CON STREAMING ===")
    print("Escribe 'salir' para terminar la conversación\\n")
    
    # Configurar asistente con personalidad
    system_message = """Eres un asistente útil y amigable especializado en tecnología. 
    Respondes de manera clara y concisa, y siempre intentas ser educativo."""
    
    while True:
        # Obtener input del usuario
        user_input = input("\\n🧑 Tú: ")
        
        if user_input.lower() in ['salir', 'exit', 'quit']:
            print("\\n👋 ¡Hasta luego!")
            break
            
        if not user_input.strip():
            continue
            
        print("\\n🤖 Asistente: ", end="", flush=True)
        
        try:
            # Streaming de la respuesta
            messages = [
                {"role": "system", "content": system_message},
                {"role": "user", "content": user_input}
            ]
            
            # Convertir a formato LangChain
            from langchain.schema import SystemMessage
            lc_messages = [
                SystemMessage(content=system_message),
                HumanMessage(content=user_input)
            ]
            
            full_response = ""
            for chunk in llm.stream(lc_messages):
                content = chunk.content
                print(content, end="", flush=True)
                full_response += content
                time.sleep(0.02)
                
            print()  # Nueva línea al final
            
        except KeyboardInterrupt:
            print("\\n\\n⏸️ Interrumpido por el usuario")
            break
        except Exception as e:
            print(f"\\n❌ Error: {e}")
            
    print("\\n¡Gracias por usar el chatbot!")

# Ejecutar chatbot (¡Pruébalo!)
chatbot_streaming() 
 # Descomenta esta línea para ejecutar

print("💡 Descomenta la línea anterior para probar el chatbot interactivo")

=== CHATBOT CON STREAMING ===
Escribe 'salir' para terminar la conversación\n


## Streaming Avanzado con Manejo de Chunks

Podemos procesar cada chunk individualmente para crear experiencias más sofisticadas.

In [9]:
# Streaming con análisis de chunks
def streaming_avanzado():
    prompt = " en 50 palabras Explica qué es la inteligencia artificial y cómo funciona el machine learning"
    
    print("=== STREAMING AVANZADO CON ANÁLISIS ===")
    print("Analizando chunks conforme llegan...\n")
    
    # Variables para estadísticas
    chunk_count = 0
    total_content = ""
    words_processed = 0
    
    try:
        for chunk in llm.stream([HumanMessage(content=prompt)]):
            chunk_count += 1
            content = chunk.content
            total_content += content
            
            # Contar palabras aproximadas
            if content.strip():
                words_in_chunk = len(content.split())
                words_processed += words_in_chunk
            
            # Mostrar progreso cada 10 chunks
            if chunk_count % 10 == 0:
                print(f"\\n[Progreso: {chunk_count} chunks, ~{words_processed} palabras]\\n")
            
            # Imprimir el contenido
            print(content, end="", flush=True)
            time.sleep(0.02)  # Pausa ligeramente más larga para ver el análisis
        
        # Estadísticas finales
        print(f"\\n\\n=== ESTADÍSTICAS FINALES ===")
        print(f"Total de chunks: {chunk_count}")
        print(f"Palabras aproximadas: {words_processed}")
        print(f"Caracteres totales: {len(total_content)}")
        print(f"Promedio chars/chunk: {len(total_content)/chunk_count if chunk_count > 0 else 0:.1f}")
        
    except Exception as e:
        print(f"\\n✗ Error: {e}")

# Ejecutar streaming avanzado
streaming_avanzado()

=== STREAMING AVANZADO CON ANÁLISIS ===
Analizando chunks conforme llegan...

La inteligencia artificial (IA) es\n[Progreso: 10 chunks, ~8 palabras]\n
 una rama de la informática que busca crear sistemas capaces\n[Progreso: 20 chunks, ~18 palabras]\n
 de realizar tareas que requieren inteligencia humana. El machine\n[Progreso: 30 chunks, ~28 palabras]\n
 learning, una subdisciplina de la IA,\n[Progreso: 40 chunks, ~38 palabras]\n
 funciona mediante algoritmos que permiten a las máquinas aprender\n[Progreso: 50 chunks, ~48 palabras]\n
 de datos y mejorar su rendimiento en tareas específicas sin\n[Progreso: 60 chunks, ~58 palabras]\n
 ser programadas explícitamente.\n\n=== ESTADÍSTICAS FINALES ===
Total de chunks: 66
Palabras aproximadas: 63
Caracteres totales: 355
Promedio chars/chunk: 5.4


## Consideraciones Técnicas del Streaming

### Cuándo Usar Streaming:
✅ **SÍ usar streaming:**
- Respuestas largas (>100 tokens)
- Aplicaciones interactivas
- Chatbots y asistentes
- Demostraciones en vivo
- Cuando la UX es prioritaria

❌ **NO usar streaming:**
- Respuestas muy cortas
- Procesamiento batch
- APIs de backend sin interfaz
- Cuando necesitas la respuesta completa antes de procesar

### Mejores Prácticas:
1. **Manejo de errores**: Siempre incluye try/catch
2. **Indicadores visuales**: Muestra progreso al usuario
3. **Cancelación**: Permite al usuario interrumpir
4. **Buffer management**: Para interfaces web, considera buffering
5. **Performance**: Monitorea el uso de recursos

## Ejercicios Prácticos

### Ejercicio 1: Indicador de Progreso
Modifica el código para mostrar un indicador de progreso (spinner, barra, porcentaje).

### Ejercicio 2: Streaming con Filtros
Implementa streaming que filtre o procese chunks específicos (ej: resaltar palabras clave).

### Ejercicio 3: Chatbot Mejorado
Extiende el chatbot con:
- Historial de conversación
- Comandos especiales (/help, /clear)
- Diferentes personalidades

## Conceptos Clave Aprendidos

1. **Streaming** mejora la percepción de velocidad
2. **Chunks** se procesan individualmente en tiempo real
3. **UX** es significativamente mejor con streaming
4. **Implementación** requiere manejo cuidadoso de generadores
5. **Casos de uso** específicos donde streaming aporta valor

## Próximos Pasos

En el siguiente notebook exploraremos la **memoria en LangChain**, que nos permite mantener contexto entre múltiples interacciones del usuario.

In [4]:
import sys
#import time
from itertools import cycle
#from langchain_openai import ChatOpenAI
#from langchain.schema import HumanMessage

def spinner_progreso():
    """Spinner que rota mientras llegan los chunks"""
    
   
    
    # Caracteres del spinner
    spinner = cycle(['⠋', '⠙', '⠹', '⠸', '⠼', '⠴', '⠦', '⠧', '⠇', '⠏'])
    
    prompt = "Explica qué es Python y sus ventajas"
    response_text = ""
    
    print("Generando respuesta...")
    
    try:
        for chunk in llm.stream([HumanMessage(content=prompt)]):
            # Mostrar spinner
            sys.stdout.write(f'\r{next(spinner)} Procesando...')
            sys.stdout.flush()
            
            response_text += chunk.content
            time.sleep(0.05)
        
        # Limpiar spinner y mostrar resultado
        sys.stdout.write('\r✅ Completado!        \n\n')
        print(response_text)
        
    except Exception as e:
        print(f"\n❌ Error: {e}")

# Ejecutar
spinner_progreso()


Generando respuesta...
✅ Completado!        

**Python** es un **lenguaje de programación** de alto nivel, interpretado, multiparadigma (soporta programación orientada a objetos, imperativa y funcional), y de propósito general. Fue creado por **Guido van Rossum** y publicado en 1991. Su diseño enfatiza la legibilidad del código y una sintaxis sencilla que permite a los programadores expresar conceptos en menos líneas de código que otros lenguajes.

### Ventajas de Python

1. **Sintaxis sencilla y legible**  
   Python está diseñado para ser fácil de leer y escribir, lo que facilita el aprendizaje y el desarrollo rápido.

2. **Gran comunidad y soporte**  
   Cuenta con una comunidad activa que aporta miles de librerías


In [5]:
import sys
import time

def barra_progreso():
    """Barra de progreso basada en caracteres generados"""
    
    prompt = "Describe el proceso de machine learning paso a paso"
    response_text = ""
    bar_length = 25
    estimated_total = 400  # Estimación de caracteres esperados
    
    print("=== Barra de Progreso ===")
    
    try:
        for chunk in llm.stream([HumanMessage(content=prompt)]):
            response_text += chunk.content
            
            # Calcular progreso
            progress = min(len(response_text) / estimated_total, 1.0)
            filled = int(bar_length * progress)
            
            # Crear barra visual
            bar = '█' * filled + '░' * (bar_length - filled)
            percent = int(progress * 100)
            
            # Actualizar barra
            sys.stdout.write(f'\r[{bar}] {percent}% ({len(response_text)} chars)')
            sys.stdout.flush()
            
            time.sleep(0.03)
        
        # Completar al 100%
        sys.stdout.write(f'\r[{"█" * bar_length}] 100% ({len(response_text)} chars)\n\n')
        print("Respuesta completa:")
        print(response_text)
        
    except Exception as e:
        print(f"\n❌ Error: {e}")

# Ejecutar
barra_progreso()


=== Barra de Progreso ===
[█████████████████████████] 100% (617 chars)

Respuesta completa:
Claro, aquí tienes una descripción paso a paso del proceso de **machine learning** (aprendizaje automático):

---

### 1. **Definición del problema**
   - Identifica y comprende el problema que deseas resolver.
   - Ejemplo: ¿Quieres clasificar correos como spam/no spam?

### 2. **Recolección de datos**
   - Obtén datos relevantes para el problema.
   - Pueden provenir de bases de datos, sensores, web, etc.

### 3. **Preprocesamiento de datos**
   - Limpieza: Elimina duplicados, corrige errores y trata valores faltantes.
   - Transformación: Normaliza o estandariza datos, convierte variables categóricas en num


In [6]:
import re

def contador_palabras():
    """Muestra el conteo de palabras conforme se genera el texto"""
    
    prompt = "Cuéntame sobre los beneficios de la inteligencia artificial"
    response_text = ""
    
    print("=== Contador de Palabras ===")
    print("Generando respuesta...\n")
    
    try:
        for chunk in llm.stream([HumanMessage(content=prompt)]):
            response_text += chunk.content
            
            # Contar palabras
            words = len(re.findall(r'\b\w+\b', response_text))
            chars = len(response_text)
            
            # Mostrar progreso
            sys.stdout.write(f'\r📝 Palabras: {words} | Caracteres: {chars}')
            sys.stdout.flush()
            
            time.sleep(0.05)
        
        print(f"\n\n✅ Generación completada!")
        print(f"📊 Estadísticas finales: {words} palabras, {chars} caracteres\n")
        print("Respuesta:")
        print(response_text)
        
    except Exception as e:
        print(f"\n❌ Error: {e}")

# Ejecutar
contador_palabras()


=== Contador de Palabras ===
Generando respuesta...

📝 Palabras: 103 | Caracteres: 727

✅ Generación completada!
📊 Estadísticas finales: 103 palabras, 727 caracteres

Respuesta:
La inteligencia artificial (IA) ofrece numerosos beneficios en diferentes ámbitos de la vida y sectores productivos. Algunos de los principales son:

1. **Automatización de tareas**: Permite que procesos repetitivos o complejos se realicen de forma automática, ahorrando tiempo y reduciendo errores humanos.

2. **Mejora en la toma de decisiones**: La IA puede analizar grandes cantidades de datos y ofrecer recomendaciones o predicciones más precisas que las realizadas manualmente.

3. **Personalización**: En áreas como el comercio electrónico, la educación y la salud, la IA puede adaptar productos, servicios o contenidos a las necesidades y preferencias individuales de los usuarios.

4. **Eficiencia operativa**: Ayuda a


In [7]:
def streaming_con_resaltado():
    """Resalta palabras clave específicas durante el streaming"""
    
    # Palabras clave a resaltar
    keywords = ["python", "machine learning", "inteligencia artificial", "algoritmo", "datos", "modelo"]
    
    prompt = "Explica qué es Python y cómo se usa en machine learning e inteligencia artificial"
    
    print("=== STREAMING CON RESALTADO DE PALABRAS CLAVE ===")
    print(f"🔍 Palabras clave: {', '.join(keywords)}\n")
    print("Respuesta (palabras clave resaltadas con *):")
    print("-" * 50)
    
    response_text = ""
    highlighted_count = 0
    
    try:
        for chunk in llm.stream([HumanMessage(content=prompt)]):
            content = chunk.content
            response_text += content
            
            # Procesar cada chunk para resaltar palabras clave
            processed_content = content
            for keyword in keywords:
                # Buscar la palabra clave (insensible a mayúsculas/minúsculas)
                pattern = re.compile(re.escape(keyword), re.IGNORECASE)
                if pattern.search(processed_content):
                    processed_content = pattern.sub(f"*{keyword.upper()}*", processed_content)
                    highlighted_count += 1
            
            # Mostrar el chunk procesado
            print(processed_content, end="", flush=True)
            time.sleep(0.05)
        
        print(f"\n{'-' * 50}")
        print(f"✅ Streaming completado!")
        print(f"🎯 Palabras clave resaltadas: {highlighted_count}")
        
    except Exception as e:
        print(f"\n❌ Error: {e}")

# Ejecutar
streaming_con_resaltado()

=== STREAMING CON RESALTADO DE PALABRAS CLAVE ===
🔍 Palabras clave: python, machine learning, inteligencia artificial, algoritmo, datos, modelo

Respuesta (palabras clave resaltadas con *):
--------------------------------------------------
***PYTHON*** es un lenguaje de programación de alto nivel, de propósito general, que se caracteriza por ser sencillo, legible y fácil de aprender. Fue creado a finales de los años 80 y desde entonces ha ganado enorme popularidad, especialmente en áreas como ciencia de *DATOS*, desarrollo web, automatización y, de manera destacada, en **machine learning (aprendizaje automático)** e **inteligencia artificial (IA)**.

### ¿Qué es *PYTHON*?

- **Sintaxis clara y sencilla**, lo que facilita el desarrollo y mantenimiento de código.
- Es **multiparadigma**, permitiendo programación orientada a objetos, imperativa y funcional.
- Posee una **gran comunidad** y una **amplia colección de bibliotecas
--------------------------------------------------
✅ Streamin